<a href="https://colab.research.google.com/github/tianshuailu/NMT-Adapt_ml_IN/blob/main/NMTAdapt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We used the same command for the first and the second iteration of training.

In [ ]:
# training
fairseq-train  /your_path/data   --encoder-normalize-before --decoder-normalize-before   --arch mbart_large --layernorm-embedding   \
--task translation_multi_simple_epoch    --criterion label_smoothed_cross_entropy --label-smoothing 0.2   --optimizer adafactor \
--lr-scheduler polynomial_decay --lr 3e-05 --min-lr -1 --warmup-updates 50 --total-num-update 80000   --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0 \
--max-tokens 512 --update-freq 8   --save-interval 1 --save-interval-updates 7000 --keep-interval-updates 10 --no-epoch-checkpoints   --seed 222 --log-format simple \
--log-interval 2  --save-dir /your_path/checkpoint \
--restore-file /your_path/pretrained_model.pt  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler   \
--ddp-backend no_c10d  --max-epoch 128  --skip-invalid-size-inputs-valid-test --memory-efficient-fp16 \
--langs ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,ml_XX,no_ML,no_HI \
--lang-pairs en_XX-hi_IN,en_XX-ml_XX,no_HI-hi_IN,no_ML-ml_XX  \
--sampling-weights "{'main:en_XX-hi_IN': 1,'main:en_XX-ml_XX': 1,'main:no_HI-hi_IN': 1,'main:no_ML-ml_XX':1}" --keep-inference-langtok --encoder-langtok src \
--decoder-langtok --lang-tok-style mbart --checkpoint-suffix pt --patience 10 

The following command is a bit different from the training command, we only used back-translation pairs for finetuning and it is only 1 epoch.

In [ ]:
# finetuning
fairseq-train  /your_path/data   --encoder-normalize-before --decoder-normalize-before   --arch mbart_large --layernorm-embedding \
--task translation_multi_simple_epoch    --criterion label_smoothed_cross_entropy --label-smoothing 0.2   --optimizer adafactor \
--lr-scheduler polynomial_decay --lr 3e-05 --min-lr -1 --warmup-updates 50 --total-num-update 80000   --dropout 0.3 --attention-dropout 0.1 --weight-decay 0.0   \
--max-tokens 512 --update-freq 2   --save-interval 1 --save-interval-updates 3000 --keep-interval-updates 10 --no-epoch-checkpoints   --seed 222 --log-format simple \
--log-interval 2  --save-dir /your_path/finetune_nmt \
--restore-file /your_path/checkpoint_lastpt.pt  --reset-optimizer --reset-meters --reset-dataloader --reset-lr-scheduler \
--ddp-backend no_c10d  --max-epoch 1  --skip-invalid-size-inputs-valid-test --memory-efficient-fp16 --langs ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,ml_XX,no_ML,no_HI \
--lang-pairs en_XX-ml_XX  --sampling-weights "{'main:en_XX-ml_XX': 1}" --keep-inference-langtok \
--encoder-langtok src --decoder-langtok --lang-tok-style mbart --checkpoint-suffix pt

We used the following command for evaluation.

In [ ]:
# model evaluation
fairseq-generate /your_path/data \
  --path /your_path/checkpoint_lastpt.pt \
  --results-path /your_path/eval_result \
  --task translation_from_pretrained_bart \
  --gen-subset test \
  -t ml_XX -s en_XX \
  --bpe 'sentencepiece' --sentencepiece-model /your_path/sentence.bpe.model \
  --scoring sacrebleu \
  --batch-size 32 --langs ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,ml_XX,no_ML,no_HI